In [ ]:
#r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
//#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Open Database

In [ ]:
BoSSSshell.WorkflowMgm.Init("CoFlowFlame_Burkkk");

Project name is set to 'CoFlowFlame_Burkkk'.
NotSupportedException caught during creation/opening of default database: `AllowedDatabasesPaths` not specified, unable to create Database (should be specified in ~/.BoSSS/etc/BatchProcessorConfig.json)..


In [ ]:
static var myBatch = BoSSSshell.ExecutionQueues[2];
// static var myBatch = GetDefaultQueue();
// static var myDb = OpenOrCreateDatabase(@"C:\Databases\BoSSSAAA");
static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\CoFlowFlame_Bur1k");
// static var myDb = myBatch.CreateOrOpenCompatibleDatabase("CoFlowFlame5");
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

Error: (5,62): error CS1003: Syntax error, ',' expected
(5,74): error CS1003: Syntax error, ',' expected
(5,74): error CS1010: Newline in constant
(5,77): error CS1003: Syntax error, ',' expected
(6,46): error CS1026: ) expected

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation()

## Case configuration

In [ ]:
int nCells = 10;
// int[] multiplierS = new int[]{2,5,11}; // multiplying factor of inlet velocities
int[] multiplierS = new int[]{1}; // multiplying factor of inlet velocities
int dgMF = 3; // Mass fraction DG degree
int[] DGDegrees = new int[] {3};

int[] nCellsArray = new int[] {8};

double[] xCellsMultiplier = new double[] {3};

double InitialMassFuelIn = 0.002400; //kg/m2s 
double InitialMassAirIn = 0.002400 *3*2; //kg/m2s 
bool parabolicVelocityProfile = false;
bool chemicalReactionActive = true; 
bool useFullGeometry = false;
bool wallBounded = true;//
bool calculateOnlyMixtureFraction = true;
double In_to_m = 2.54/100;

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid(int nCells) {
        // Geometry
        // All lenghts are non.dimensionalized with Lref = fuel inlet radius (0.2 cm)
        double factor = 1.0;
        double In_to_m = 2.54/100;

        double r = (1.0/6.0)*In_to_m; // Radius inner cylinder, m
        double R = 1*In_to_m; // Radius outter cylinder, m
        double LRef =1.0;// r;

        double zlength = 20.0 / 100 * 1;// meters
        double xmin = -R / LRef;
        double xmax = +R / LRef;
        double ymin = 0;
        double ymax = zlength / LRef;
        double rAd = r / LRef;

        double leftmidpoint = (xmin - rAd) / 2;
        double rightmidpoint = (xmax + rAd) / 2;

        var troubledPoints = new double[][] {
        new double[]{ rAd, 0.0 },
        new double[]{-rAd, 0.0 }
        };
        double stretchfactorY = 0.98 * 1;

        double sf1 = 0.97 * 1 * 0;
        double sf2 = 0.95 * 1 * 0;
        double sf3 = 0.97 * 1 * 0;
        int n1 = (int)2.5 * nCells;
        int n2 = (int)1.0 * nCells;
        int n3 = (int)2.5 * nCells;
        //var xNodes1 = GenericBlas.SinLinSpacing(xmin, -rAd, sf1, n1 + 1);
        //var xNodes2 = GenericBlas.SinLinSpacing(-rAd, rAd, sf2, n2 + 1);
        //var xNodes3 = GenericBlas.SinLinSpacing(rAd, xmax, sf3, n3 + 1);

        List<double> xNodes2 = (GenericBlas.SinLinSpacing(-rAd, rAd, sf2, n2 + 1)).ToList(); // nodes corresponding to the fuel inlet
        List<double> xNodes3 = (GenericBlas.SinLinSpacing(rAd, (xmax - rAd) * 2 + rAd, sf3, n1 * 2 + 1).ToList()); // Nodes corresponding to the oxidizer inlet, right part
        var myXnodes3 = xNodes3.GetSubVector(0, xNodes3.Count / 2 + 1); // Take only "left side" of node array
        var myxNodes1 = myXnodes3.CloneAs();
        myxNodes1.ScaleV(-1.0);
        Array.Reverse(myxNodes1);

        List<double> list2 = new List<double>();
        list2.AddRange(myxNodes1.Take(n1 + 0).ToList());
        list2.AddRange(xNodes2.Take(n2 + 0).ToList());
        list2.AddRange(myXnodes3.Take(n3 + 1).ToList());
        double[] _xNodes = list2.ToArray();
        //Debug.Assert(_xNodes.Contains(xmin));
        //Debug.Assert(_xNodes.Contains(-rAd));
        //Debug.Assert(_xNodes.Contains(rAd));
        //Debug.Assert(_xNodes.Contains(xmax));
        var _yNodes = GenericBlas.SinLinSpacing(ymin, ymax * 2, stretchfactorY, (2 * nCells) * 4 + 1);
        var myYnodes = _yNodes.GetSubVector(0, _yNodes.Length / 2 + 1); // I just want a fine mesh in the bottom part of the grid.
                                                                        //var _xNodes = GenericBlas.Linspace(xmin, xmax, 2 * meshScaling + 1);
                                                                        //var myYnodes = GenericBlas.Linspace(ymin, ymax, 2 * meshScaling + 1);
        var grd = Grid2D.Cartesian2DGrid(_xNodes, myYnodes, periodicX: false);
        grd.EdgeTagNames.Add(1, "Velocity_Inlet_O2");
        grd.EdgeTagNames.Add(2, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");
        //grd.EdgeTagNames.Add(4, "wall");
        grd.EdgeTagNames.Add(4, "NoSlipNeumann");
        // grd.EdgeTagNames.Add(4, "Velocity_Inlet_outer"); // We want a constant velocity field.

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //Edge tags
            //1: Velocity inlet O_2
            //2: Velocity inlet CH_4
            //3: Pressure outlet

            //Inlet fuel
            if(Math.Abs(y - ymin) < 1e-8 && Math.Abs(x - 0.0) < rAd + 1e-8)
                return 2;
            //Inlet oxidizer
            if((Math.Abs(y - ymin) < 1e-8 && ((Math.Abs(x - rightmidpoint) < ((xmax - rAd) * 0.5 + rAd) + 1e-8) || Math.Abs(x - leftmidpoint) < Math.Abs((xmin - rAd) * 0.5 + rAd) + 1e-8)))
                return 1;

            // Pressure outlet

            if((Math.Abs(y - ymax) < 1e-8))
                return 3;

            // Pressure outlet

            if((Math.Abs(x - xmin) < 1e-8) || (Math.Abs(x - xmax) < 1e-8))
                return 4;
            else throw new ArgumentOutOfRangeException();
        
        }
        );
        myDb.SaveGrid(ref grd);

        return grd;
    }
}

Error: (96,9): error CS0103: The name 'myDb' does not exist in the current context

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma) {
        using(var stw = new System.IO.StringWriter()) {

            stw.WriteLine("static class BoundaryValues {");

            stw.WriteLine("  static public double ConstantValue(double[] X) {");
            stw.WriteLine("    return "+ ConstVal +";");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuel(double[] X) {");
            stw.WriteLine("    return  (1.0 - Math.Pow(X[1] / "+inletRadius+", 2)) * "+uInFuel+" ;");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityAir(double[] X) {");
            stw.WriteLine("    return  -(1.0 - Math.Pow(X[1] / "+inletRadius+", 2)) * "+uInAir+";");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double RegularizedPlugFlowFuel(double[] X) {");    
            stw.WriteLine(" double res = 0;");  
            stw.WriteLine("         if(X[1] > 0) { ");
            stw.WriteLine("     double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] - "+inletRadius+"))); ");
            stw.WriteLine("           res = "+uInFuel+" * (1 - 2*H); ");
            stw.WriteLine("      }   ");
            stw.WriteLine("  else { ");
            stw.WriteLine("      double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] + ("+inletRadius+")))); ");
            stw.WriteLine("           res = "+uInFuel+" * ( 2*H-1); ");
            stw.WriteLine("  } ");
            stw.WriteLine("return res;"); 
            stw.WriteLine("  }");


            stw.WriteLine("  static public double RegularizedPlugFlowAir(double[] X) {");         
            stw.WriteLine(" double res = 0;");
            stw.WriteLine("         if(X[1] > 0) { ");
            stw.WriteLine("     double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] - "+inletRadius+"))); ");
            stw.WriteLine("           res = "+uInAir+" * (1 - 2*H)*(-1); ");
            stw.WriteLine("      }   ");
            stw.WriteLine("  else { ");
            stw.WriteLine("      double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] + ("+inletRadius+")))); ");
            stw.WriteLine("           res = "+uInAir+" * ( 2*H-1)*(-1); ");
            stw.WriteLine("  } ");
            stw.WriteLine("return res;"); 
            stw.WriteLine("  }");


            stw.WriteLine("}"); 
            return stw.ToString();
        }
    }

    static public Formula Get_ConstantValue(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ConstantValue", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_ParabolaVelocityFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }
    static public Formula Get_ParabolaVelocityAir(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityAir", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_RegularizedPlugFlowFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.RegularizedPlugFlowFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_RegularizedPlugFlowAir(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.RegularizedPlugFlowAir", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

}

## Create base control file
In this ControlFile basic configuration of the CounterDiffusionFlame is defined.

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int nCells, bool isMF, double massFuelIn, double massAirIn, bool parabolicVelocityProfile, bool UsefullGeometry, bool wallBounded, double xNodesMultiplier) {
    var CC = new ChemicalConstants();

    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();

    C.NumberOfChemicalSpecies = 4; 
    C.SetDGdegree(dg); //
    C.SetGrid(GridFactory.GenerateGrid(nCells));  //

    C.MatParamsMode = MaterialParamsMode.Constant;  //
    // Problem Definition
    //===================
    double TemperatureInOxidizer = 300; //
    double AtmPressure = 101325; // Pa
    // double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };
    double[] FuelInletConcentrations = new double[] { 1.0, 0.0, 0.0, 0.0, 0.0 };

    // double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; //
    // double[] OxidizerInletConcentrations = new double[] { 0.0, 0.5, 0.0, 0.0, 0.5 }; //
    double[] OxidizerInletConcentrations = new double[] { 0.0, 1.0, 0.0, 0.0, 0.0 }; //

    double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };





    // Reference values
    //===================
    // Basic units to be used: Kg, m, s, mol, pa,
    double In_to_m = 2.54/100;
    double TRef = TemperatureInOxidizer;// Reference temperature  is the inlet temperature, (K)
    double pRef = AtmPressure; // Pa
    double uIn = 1.33 * In_to_m ;
    double uRef =1.0;//uIn ;

    double r = (1.0/6.0)*In_to_m; // Radius inner cylinder, m
    double R = 1*In_to_m; // Radius outter cylinder, m
    double LRef = 1.0;//r;

    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 }; //No gravity.

    // Solver configuration
    // =======================
    C.rhoOne = true;//!!!!

    C.smoothingFactor = 80;
    // C.NonLinearSolver.ConvergenceCriterion = 1e-8;
    // C.LinearSolver.ConvergenceCriterion = 1e-10;
    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.LinearSolver.SolverCode = LinearSolverCode.classic_pardiso;
    C.LinearSolver.verbose = false;
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.saveperiod = 1;
    C.PenaltyViscMomentum = 1.0; 
    C.PenaltyHeatConduction = 1.0;
    C.YFuelInlet = FuelInletConcentrations[0];
    C.YOxInlet = OxidizerInletConcentrations[1];
    C.FuelInletConcentrations = FuelInletConcentrations;
    C.OxidizerInletConcentrations = OxidizerInletConcentrations;
    C.TFuelInlet = 1.0;
    C.TOxInlet = 1.0;
    C.PhysicalParameters.IncludeConvection = true;
    // Chemical related parameters
    double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
    C.phi = s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
    Console.WriteLine("zSt is: "+C.zSt);
    var ThermoProperties = new ThermodynamicalProperties();

    //==========================
    //Derived reference values
    //==========================
    C.uRef = 1.0;//uRef; // Reference velocity
    C.LRef = 1.0;//LRef; // reference length
    C.pRef = AtmPressure; // reference pressure
    C.TRef = 300;// reference temperature
    C.MWRef =1.0;
    C.rhoRef = C.pRef * C.MWRef / (8.314 * C.TRef * 1000); // Kg/m3. ok ;
    C.cpRef = -10000;// not important
    C.muRef = 4.9226e-05; // mu = rho D = k/cp because Pr = Le = 1.0
    C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
    C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses
    C.T_ref_Sutherland = 300;
    double heatRelease_Ref = (C.TRef * C.cpRef);
    C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
    C.B = CC.PreExponentialFactor;

    C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };

    C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef);
    C.Reynolds =  C.uRef * C.LRef / C.muRef;
    C.Prandtl = 1.0;
    C.Schmidt = C.Prandtl; // Because Lewis number  is assumed as 1.0  (Le = Pr/Sc)
    C.Lewis = new double[] { 1.0, 1.0, 1.0, 1.0, 1.0 };

    double g = 9.8; // m/s2
    C.Froude = Math.Sqrt(uRef * uRef / (C.LRef * g)); // Not used
    C.T_ref_Sutherland = 300; //////// Check this
    C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 }; 
    //==========================
    // Initial conditions
    //==========================
    double dummy = 0;
    double Radius = 0.5;
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uIn / C.uRef, uIn / C.uRef,dummy));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uIn / C.uRef, uIn / C.uRef,dummy));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uIn / C.uRef, uIn / C.uRef,dummy));

    //==========================
    // Boundary conditions
    //==========================
    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue( uIn / uRef, dummy, dummy, dummy, dummy));



    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue( uIn / uRef, dummy, dummy,dummy, dummy));

    // C.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy,dummy, dummy));
    // C.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue( uIn / uRef, dummy, dummy,dummy, dummy));

    C.AddBoundaryValue("Pressure_outlet");
    return C;
}

Error: (8,15): error CS0103: The name 'GridFactory' does not exist in the current context

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn, bool parabolicVelocityProfile, int multiplier, bool useFullGeometry, bool wallBounded, double xNodesMultiplier){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true,massFuelIn, massAirIn, parabolicVelocityProfile, useFullGeometry,  wallBounded , xNodesMultiplier);
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "CoFlowFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells + "Multiplier" +multiplier + "xNodesMultiplier"+xNodesMultiplier;
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.UseSelfMadeTemporalOperator = false;
    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 50; 
    C_MixtureFraction.dummycounter = multiplier;
    
    // Boundary and initial conditions
    double dummy = -11111111;
    C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(0.0,dummy,dummy , dummy, dummy));
    // C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(0.0,dummy,dummy , dummy, dummy));

    C_MixtureFraction.Tags.Add(multiplier.ToString()); // Tag used for restart of the full problem
    double radius_inlet = 1;
    var troubledPoints = new List<double[]>();
 
        troubledPoints.Add(new double[]{1, 0});
        troubledPoints.Add(new double[]{-1, 0});
        

    C_MixtureFraction.AdaptiveMeshRefinement = false;
    C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 

    // C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onProblematicPoints(troubledPoints,C_MixtureFraction.AMR_startUpSweeps) );  
    // C_MixtureFraction.activeAMRlevelIndicators.Add(new BoSSS.Application.XNSEC.AMR_RefineAroundProblematicPoints(troubledPoints, 1, 0.01));
    // C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_MixtureFraction.zSt,3) );
    return C_MixtureFraction;
}

Error: (2,29): error CS0103: The name 'GiveMeTheCtrlFile' does not exist in the current context

## Send and run jobs

In [ ]:
foreach(double xDensityMult in  xCellsMultiplier)
foreach(int nCells in nCellsArray){

foreach(int i in multiplierS){
  double massFuelIn = InitialMassFuelIn*i;
  double massAirIn = InitialMassAirIn*i;

  Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
  var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dgMF, nCells, massFuelIn, massAirIn,  parabolicVelocityProfile, i, useFullGeometry, wallBounded, xDensityMult);
  Console.WriteLine("C_MixtureFraction.zSt" + C_MixtureFraction.zSt);
  Console.WriteLine("Reynolds" + C_MixtureFraction.Reynolds);
  string jobName       = C_MixtureFraction.SessionName;
  Console.WriteLine(jobName);
  var oneJob           = new Job(jobName, solver_MF);
  oneJob.NumberOfMPIProcs =  8;
  oneJob.SetControlObject(C_MixtureFraction);
  oneJob.Activate(myBatch); 
}
}

Error: (9,27): error CS0103: The name 'GiveMeTheMixtureFractionCtrlFile' does not exist in the current context
(17,19): error CS0103: The name 'myBatch' does not exist in the current context

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

All jobs finished.


## Starting the finite-rate chemistry simulation

Now that the simulation for an "infinite" reaction rate is done, we use it for initializing the system with finite reaction rate. The goal is to obtain solutions of the counter difussion flame for increasing strain values. We start with a low strain (bigger Dahmkoehler number), which is increased until extintion is (hopefully) found

In [ ]:
static XNSEC_Control GiveMeTheFullCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn, ISessionInfo SessionToRestart, bool parabolicVelocityProfile, bool chemReactionActive, bool useFullGeometry, bool wallBounded, int mult, int counter, double xNodesMultiplier) {
    var C_OneStep = GiveMeTheCtrlFile(dg, nCells, false, massFuelIn, massAirIn, parabolicVelocityProfile, useFullGeometry, wallBounded, xNodesMultiplier);
    C_OneStep.physicsMode = PhysicsMode.Combustion;
    C_OneStep.ProjectName = "CoFlowFlame";
    string name = C_OneStep.ProjectName + "P" + dg + "K" + nCells + "mult" + mult+"_c"+counter ;
    C_OneStep.SessionName = "Full_" + name;
    C_OneStep.VariableOneStepParameters = false;

    // C_OneStep.Tags.Add("VelocityMultiplier" + mult);
    C_OneStep.Tags.Add( mult.ToString());
    C_OneStep.dummycounter = counter;
    C_OneStep.UseSelfMadeTemporalOperator = false;
    C_OneStep.myThermalWallType = SIPDiffusionTemperature.ThermalWallType.Adiabatic;
    C_OneStep.Timestepper_LevelSetHandling = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    C_OneStep.UseMixtureFractionsForCombustionInitialization = true;

    C_OneStep.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
    C_OneStep.LinearSolver.NoOfMultigridLevels = 5;
    C_OneStep.ChemicalReactionActive = chemReactionActive;
    C_OneStep.AdaptiveMeshRefinement = true;
    
    C_OneStep.HeatCapacityMode = MaterialLaw_MultipleSpecies.CpCalculationMode.mixture;

    bool AMRinEachNewtonStep = false;

    if( AMRinEachNewtonStep) {
        C_OneStep.NoOfTimesteps = 4;
        C_OneStep.NonLinearSolver.MaxSolverIterations = 8; // Do only one newton iteration before refining
        C_OneStep.NonLinearSolver.MinSolverIterations = 8; // Do only one newton iteration before refining
    } else{
        // C_OneStep.NoOfTimesteps = 1; // The steady solution will be calculated again and do AMR
        C_OneStep.NonLinearSolver.MaxSolverIterations = 30;
    }

    C_OneStep.AMR_startUpSweeps = 0;
    if(C_OneStep.ChemicalReactionActive){
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_onReactiveZones(C_OneStep.MolarMasses, 2, 0.2));
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnVariableLimits("Temperature", new double[] { -100, 4 },3)); // Refine all cells with T > 5 (and T < -100)
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnFieldGradient(2, 0.9, VariableNames.Temperature));
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 2));
    }
    // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 3));
    // C_OneStep.NonLinearSolver.MaxSolverIterations = 10;

    // limiting of variable values
    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    double eps = 1e-2;
    Bounds.Add(VariableNames.Temperature, new Tuple<double, double>(1.0 - eps, 10)); // Min temp should be the inlet temperature.
    Bounds.Add(VariableNames.MassFraction0, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4)); // Between 0 and 1 per definition
    Bounds.Add(VariableNames.MassFraction1, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction2, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction3, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    C_OneStep.VariableBounds = Bounds;
    // Boundary conditions

    double dummy = 0;
    if(SessionToRestart != null) {
        C_OneStep.SetRestart(SessionToRestart);
    } else {
        C_OneStep.AddInitialValue(VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(0.23, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    }
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[0], dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[1], dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[2], dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[3], dummy, dummy,dummy, dummy));

    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[0], dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[1], dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[2], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[3], dummy, dummy,dummy, dummy));

    C_OneStep.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[0], dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[1], dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[2], dummy, dummy,dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[3], dummy, dummy, dummy, dummy));
    return C_OneStep;
}

Error: (2,21): error CS0103: The name 'GiveMeTheCtrlFile' does not exist in the current context

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);

Calculate the full solution for the initial value

In [ ]:
int counter = 0;
if(!calculateOnlyMixtureFraction){
foreach (double xDensityNodes in xCellsMultiplier){
    foreach (int nCells in nCellsArray) {
        foreach (int dg in DGDegrees) {
            foreach (int i in multiplierS) {
                var sess = (myDb.Sessions.Where(s => s.Name == "FS_CoFlowFlameP" + dgMF + "K" + nCells + "Multiplier" + i + "xNodesMultiplier" + xDensityNodes)).FirstOrDefault();
                if(sess != null){
                    Console.WriteLine("Restarting from session with name " + sess.Name); 
                } else{
                    Console.WriteLine("No session found for restart. Starting simulation without combustion");
                }
                var C = GiveMeTheFullCtrlFile(dg, nCells, InitialMassFuelIn * i, InitialMassAirIn * i, sess, parabolicVelocityProfile, chemicalReactionActive, useFullGeometry, wallBounded, i, counter, xDensityNodes);
                string jobName = C.SessionName;
                Console.WriteLine(jobName);
                var oneJob = new Job(jobName, solver);
                oneJob.NumberOfMPIProcs = 12;
                oneJob.SetControlObject(C);
                oneJob.Activate(myBatch);
                counter++;
            }
        }
    }
}
}

Error: (7,29): error CS0103: The name 'myDb' does not exist in the current context
(13,25): error CS0103: The name 'GiveMeTheFullCtrlFile' does not exist in the current context
(19,33): error CS0103: The name 'myBatch' does not exist in the current context

In [ ]:
// wait for all jobs to finish (up to 2 days, check every 10 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*2), PollingIntervallSeconds:(60*10));

All jobs finished.


In [ ]:
// detect failed Jobs in the job management
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects

In [ ]:
NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

In [ ]:
// // Delete all MF calculations
// Console.WriteLine("Deleting mass fraction calculations");
// myDb.Sessions.Where(sess => sess.Name.Contains("FS")).ForEach(sess=> sess.Delete(true))

In [ ]:
myDb.Sessions[0].Export().Do()

Error: (1,1): error CS0103: The name 'myDb' does not exist in the current context